In [10]:
# load data with bounds
#########

from astropy.table import Table
from astropy.coordinates import SkyCoord

previous_filename = "gd1_redux_02.hdf5"
cm_polygon_t = Table.read(previous_filename, path="cm_polygon")
pm_vertices_t = Table.read(previous_filename, path="pm_vertices")

#icrs_corners_t = Table.read(previous_filename, path="ircs_corners")
#icrs_corners=SkyCoord(icrs_corners_t['ra'], icrs_corners_t['dec'], frame="icrs")


In [11]:
# actually icrs_corners is not being saved/loaded right so we'll just recompute it here
#######

from common import make_rectangle
import astropy.units as u
from gala.coordinates import GD1Koposov10
from gala.coordinates import reflex_correct

# create a series of rectangles for the query 
phi1_ranges = [(-100, -70), (-70, -40), (-40, -10), (-10, 20)] *u.deg
phi2_range = ( -8, 4)*u.deg

# make the rectangle for each range
rects = map(lambda phi1_range: make_rectangle(phi1_range, phi2_range), phi1_ranges)
# and reformat from a list of tuples to tuple of lists
rects = zip(*rects)
rects = tuple(map(list, rects))

# convert from the GD1 to ICRS frame
gd1_frame = GD1Koposov10()
gd1_corners = SkyCoord(phi1=rects[0], phi2=rects[1], frame=gd1_frame)
icrs_corners = gd1_corners.transform_to('icrs')


In [12]:
column_list = ['gaia.source_id',
               'gaia.ra',
               'gaia.dec',
               'gaia.pmra',
               'gaia.pmdec',
               'best.number_of_neighbours',
               'best.number_of_mates',
               'ps.g_mean_psf_mag',
               'ps.i_mean_psf_mag']

columns = ", ".join(column_list)

query_base = """SELECT
    {columns}
    FROM gaiadr3.gaia_source as gaia
    JOIN gaiadr3.panstarrs1_best_neighbour as best
        ON gaia.source_id = best.source_id
    JOIN gaiadr2.panstarrs1_original_valid as ps
        ON best.original_ext_source_id = ps.obj_id

    WHERE gaia.parallax < 1
        AND gaia.bp_rp BETWEEN -0.75 AND 2
        AND 1 = CONTAINS(
            POINT(gaia.ra,gaia.dec),
            POLYGON({radec_vertices})
        )
        AND 1 = CONTAINS(
            POINT(gaia.pmra,gaia.pmdec),
            POLYGON( {pm_vertices} )
        )
"""




In [13]:

from astropy.table import Table, vstack, unique
from common import skycoords_to_string, table_to_string
from astroquery.gaia import Gaia

# accumulator table for all results
results_t = Table()

# for each corners range identified, create and execute a query
for region in icrs_corners:
    query = query_base.format(columns=columns, radec_vertices=skycoords_to_string(region), pm_vertices=table_to_string(pm_vertices_t))
    print(query)
    job = Gaia.launch_job_async(query)
    results = job.get_results()
    # accumuate the results
    results_t = vstack([results_t, results])
    results_t = unique(results_t, keys="source_id")


SELECT
    gaia.source_id, gaia.ra, gaia.dec, gaia.pmra, gaia.pmdec, best.number_of_neighbours, best.number_of_mates, ps.g_mean_psf_mag, ps.i_mean_psf_mag
    FROM gaiadr3.gaia_source as gaia
    JOIN gaiadr3.panstarrs1_best_neighbour as best
        ON gaia.source_id = best.source_id
    JOIN gaiadr2.panstarrs1_original_valid as ps
        ON best.original_ext_source_id = ps.obj_id

    WHERE gaia.parallax < 1
        AND gaia.bp_rp BETWEEN -0.75 AND 2
        AND 1 = CONTAINS(
            POINT(gaia.ra,gaia.dec),
            POLYGON(123.041, -19.002, 112.264, -12.9365, 127.403, 12.9524, 137.893, 6.84216, 123.041, -19.002)
        )
        AND 1 = CONTAINS(
            POINT(gaia.pmra,gaia.pmdec),
            POLYGON( 3.46874164e+02,  -2.83569068, 3.47201944e+02,  -2.47661737, 3.49318861e+02,  -2.04342333, 3.50522538e+02,  -2.23517877, 3.50593343e+02,  -2.40935599, 3.50704532e+02,  -3.50791024, 3.50711640e+02,  -3.54837906, 3.51062532e+02,  -4.94266179, 3.51564749e+02,  -6.09525585, 

In [14]:
# add GD1 coordinates to each row
######

from gala.coordinates import GD1Koposov10
from gala.coordinates import reflex_correct
from common import icrs_to_gd1

# the GD1 stars are too far away for accurate parallax, so use a approximate average value for distance
distance = 8 * u.kpc
radial_velocity = 0 * u.km/u.s

# create coordinate objects for each entry in the table
skycoord_icrf = SkyCoord(ra=results_t['ra'], 
                    dec=results_t['dec'],
                    pm_ra_cosdec=results_t['pmra'],
                    pm_dec=results_t['pmdec'], 
                    distance=distance, 
                    radial_velocity=radial_velocity)

# create coresponding gd1 frame coordinate objects
skycoord_gd1 = icrs_to_gd1(skycoord_icrf)

# add these coordinates back to the results table
results_t['phi1'] = skycoord_gd1.phi1
results_t['phi2'] = skycoord_gd1.phi2
results_t['pm_phi1'] = skycoord_gd1.pm_phi1_cosphi2
results_t['pm_phi2'] = skycoord_gd1.pm_phi2 

results_t

source_id,ra,dec,pmra,pmdec,number_of_neighbours,number_of_mates,g_mean_psf_mag,i_mean_psf_mag,phi1,phi2,pm_phi1,pm_phi2
,deg,deg,mas / yr,mas / yr,,,,mag,deg,deg,mas / yr,mas / yr
int64,float64,float64,float64,float64,int16,int16,float64,float64,float64,float64,float64,float64
576555035425747840,134.35628990005787,0.6249678350154616,-0.4486304353659947,-11.974387562201748,1,0,19.353099822998,17.5522003173828,-77.21836778285393,-8.149050190571844,-6.363145492303263,-3.299712527861567
576743124929516288,134.31449043817358,0.6693710170100605,-4.249951051121263,-8.704192849291294,1,0,20.2045993804932,18.4395008087158,-77.20081324835445,-8.090598495084222,-5.445365303006253,1.6344910732633033
576744117065775488,134.32676048233915,0.7190757882617109,-5.932986469487561,-7.975171627817813,1,0,19.1553001403809,17.4416007995605,-77.1511845471864,-8.07621837367769,-5.6577935646981175,3.4562778936620053
576744460663151360,134.4024761752294,0.6921142291829124,-4.356012103421504,-8.562803006665305,1,0,22.0433006286621,19.843900680542,-77.13627649233604,-8.155219017019448,-5.371877859649964,1.791319916451459
576744838620281856,134.38065702164064,0.7207908329469188,-6.306286443110489,-8.735234931488952,1,0,19.5247993469238,18.256799697876,-77.12231790577704,-8.121941194527638,-6.500581884024248,3.39279791837473
576745216577417216,134.45131134169452,0.752321902909889,-2.7260980587820978,-10.675007455843922,1,0,18.2208995819092,17.8376998901367,-77.058895642283,-8.167149211415111,-6.372202964377343,-0.6818323670316968
576745495751441408,134.41993780773427,0.7723104073632779,-2.3788982675827826,-10.08592848781956,1,0,17.6303005218506,17.1047992706299,-77.05738270134407,-8.129981729687419,-5.6883347264780815,-0.6827657816385133
576746144290354816,134.37346478332796,0.7573525638871151,-10.24256705848228,-3.0852365111353355,1,0,20.7219009399414,18.7863006591797,-77.09404852631997,-8.097339865444281,-3.5945577241815347,9.637843232666416


In [15]:
# save all that beautiful data
#######

len(results_t)
filename = "gd1_redux_03.hdf5"
results_t.write(filename, path="results", serialize_meta=True, overwrite=True)
